In [1]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
import warnings
from OutilsP5 import *

#SciKit-Learn
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans


from sklearn.metrics import silhouette_score
from sklearn.metrics import adjusted_rand_score


In [2]:
# On ingnorera les warning concernant les prochaines versions
def warn(*args, **kwargs):
    pass
warnings.warn = warn

## <font color='green'> Import des données </font> 

Lancer le notebook d'exploration afin d'obtenir le fichier modelData.csv & orders_view.csv.

In [3]:
orders_view = pd.read_csv('orders_view.csv')
data = pd.read_csv("modelData.csv")
data.set_index("customer_unique_id", inplace = True)
data.head(3)

,recency,frequency,monetary,satisfaction
customer_unique_id,,,,
0000366f3b9a7992bf8c76cfdf3221e2,115,1,130,5.0
0000b849f77a49e4a4ce2b2a4ca5be3f,118,1,19,4.0
0000f46a3911fa3c0805444483337064,541,1,69,3.0


In [4]:
normalizer = StandardScaler()
model_data = pd.DataFrame(normalizer.fit_transform(data), index = data.index, columns = data.columns)

# <font color='red'> Application du contrat de maintenance au K-means </font>

In [ ]:
all_X = []
all_Y = []

for j in  [70,140,210,280,350] :
    print(j)
    X = []
    Y = []
    orders_view['purchase_date'] = pd.to_datetime(orders_view['purchase_date'], format = "%Y-%m-%d %H:%M:%S")
    today = orders_view["purchase_date"].max()
    maj_date = today-dt.timedelta(days = j)
    old_data, customers_list = timeCut(orders_view, end_date = maj_date)

    # Prédiction des nouveaux clients sur la base des données précédentes
    oldModelKMeans = KMeans(n_clusters = 5)
    old_model_data = pd.DataFrame(normalizer.fit_transform(old_data), index = old_data.index, columns = old_data.columns)
    oldModelKMeans.fit(old_model_data)


    for i in range(70) :
        new_data, customers_list = timeCut(orders_view, end_date = maj_date + dt.timedelta(days = i))
        new_model_data = normalizer.fit_transform(new_data)
        kmeans_prediction = oldModelKMeans.predict(new_model_data)
    
        # Prédiction des nouveaux clients sur la base de données mise-à-jour
        new_model_data = pd.DataFrame(normalizer.fit_transform(new_data), index = new_data.index, columns = new_data.columns)
        newModelKMeans = KMeans(n_clusters = 5)
        newModelKMeans.fit(new_model_data)
    
        df = new_model_data.copy()
        df['label'] = newModelKMeans.labels_
        df['predicted_label'] = kmeans_prediction
        X.append(i)
        Y.append(adjusted_rand_score(df['label'], df['predicted_label']))
    
    all_X.append(X)
    all_Y.append(Y)
X_df = pd.DataFrame(all_X)
Y_df = pd.DataFrame(all_Y)
plt.plot(X_df.mean(axis = 0),Y_df.mean(axis = 0))
plt.title("Conservation de la performance dans le temps")
plt.xlabel("Jours écoulés après une mise-à-jour")
plt.ylabel("Performance conservée")

70
140


In [ ]:
plt.figure(figsize = (13,7))
plt.plot(X_df.mean(axis = 0),Y_df.mean(axis = 0))
plt.title("Conservation de la performance dans le temps")
plt.xlabel("Jours écoulés après une mise-à-jour")
plt.ylabel("Performance conservée")